# Homework \#2

Alles Rebel

Computional Science PhD

## Julia Package Requirements

Run these cells before anything else just to ensure the julia environment has all the dependencies it needs!

In [1]:
import Pkg; 
Pkg.add("Distributions")
Pkg.add("PrettyTables")
Pkg.add("Optim")

using Printf
using LinearAlgebra
using ForwardDiff
using Random
using Distributions
using Statistics
using Dates
using PrettyTables
using Optim
using Plots

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


## Problem 1, Multi-Variable Unconstrained Optimization!

Problem 1 is all about gradient based optimization. We'll have access to derivatives staying at first order only. We'll look at two methods, a regular gradient descent method and a conjugate gradient descent method. In both cases we'll instrument and compare them!

Similiar the the previous homeworks, we'll be using the following template from K&W algorithm 5.1 for gradabilty:

In [2]:
## Original K&W algorithm 5.1 Code! 
# It's a functional gradient descent optimizer... but we'll add stuff soon
abstract type DescentMethod end

struct GradientDescent <: DescentMethod
    α
end

init!(M::GradientDescent, f, ∇f, x) = M

function step!(M::GradientDescent, f, ∇f, x)
    α, g = M.α, ∇f(x)
    return x - α*g
end

step! (generic function with 1 method)

### Problem 1A

#### Quick Code Explaination!
The template code is actually a functional implementation!
A quick run down: $DescentMethod$ defines a interface. A immutable struct is declared, that inherits the DescentMethod interface, including the parameters that are needed for specifically $GradidentDescent$. The struct contains a single value, a $\alpha$ learning rate. 

As mentioned by the text, there's an inline function called $init!$. It's a function doesn't actually do anything meaningful - all it does is take the passed in parameter $M$ and return it. It does specify that the parameter should be a instance of $GradientDescent$ but beyond that, it's not super useful (yet).

The meat and bones in $step!$, which is where we'll be implementing the actual logic for the gradient descent. The next value of $x$ is returned, subtracting off the gradient direction times the learning rate defined in the $M$ parameter passed in. An example usage is shown below of the template code!

In [3]:
# Example usage of the K&W algorithm 5.1 Code! 
f(x) = (x - 3)^2 # an example! root = 3
∇f(x) = 2 * (x - 3) # the derivative

M = GradientDescent(0.1)
x = 0.0  # Initial guess, we'll make this random soon

for i in 1:20
    x = step!(M, f, ∇f, x)
    @printf("Iteration %d: x = %2.7f, f(x) = %2.7f\n", i, x, f(x))
end

@printf("Found solution: x ≈ %2.7f\n", x)

Iteration 1: x = 0.6000000, f(x) = 5.7600000
Iteration 2: x = 1.0800000, f(x) = 3.6864000
Iteration 3: x = 1.4640000, f(x) = 2.3592960
Iteration 4: x = 1.7712000, f(x) = 1.5099494
Iteration 5: x = 2.0169600, f(x) = 0.9663676
Iteration 6: x = 2.2135680, f(x) = 0.6184753
Iteration 7: x = 2.3708544, f(x) = 0.3958242
Iteration 8: x = 2.4966835, f(x) = 0.2533275
Iteration 9: x = 2.5973468, f(x) = 0.1621296
Iteration 10: x = 2.6778775, f(x) = 0.1037629
Iteration 11: x = 2.7423020, f(x) = 0.0664083
Iteration 12: x = 2.7938416, f(x) = 0.0425013
Iteration 13: x = 2.8350733, f(x) = 0.0272008
Iteration 14: x = 2.8680586, f(x) = 0.0174085
Iteration 15: x = 2.8944469, f(x) = 0.0111415
Iteration 16: x = 2.9155575, f(x) = 0.0071305
Iteration 17: x = 2.9324460, f(x) = 0.0045635
Iteration 18: x = 2.9459568, f(x) = 0.0029207
Iteration 19: x = 2.9567654, f(x) = 0.0018692
Iteration 20: x = 2.9654124, f(x) = 0.0011963
Found solution: x ≈ 2.9654124


#### Enhancements...

We'll want to keep track of a few things:
* How many function calls/Number of Iterations?
* Tolerance? Maybe we get to the solution early! 

In [4]:
## Updated K&W algorithm 5.1 Code! 
# Everything else is the same, just updated the step! function with basic instrumentation
# Enhanced with function call instrumentation + early exit based on tolerance
# the interface is the same as other DescentMethods
mutable struct MyGradientDescent <: DescentMethod
    α::Float64

    # instrumention stuff
    func_calls::Int
    grad_calls::Int
    iterations::Int
    converged::Float64

    # Constructor with optional instrumentation fields defaulting to zero/false
    function MyGradientDescent(α::Float64)
        new(α, 0, 0, 0, 0.0)
    end
end

function init!(M::MyGradientDescent, f, ∇f, x)
    # instrumentation init, since we want to reuse the M
    M.func_calls = 0
    M.grad_calls = 0
    M.iterations = 0
    M.converged = 0.0
end

function step!(M::MyGradientDescent, f, ∇f, x; tol=1e-6)
    # early exit check
    if M.converged > 0.0
        return x
    end
    
    # we're about to actually do an iteration/step
    M.iterations += 1
    
    α, g = M.α, ∇f(x)
    M.grad_calls += 1
    
    # tolerance check, maybe we're done! If the magnitude / norm of the gradient is small
    # we're probably in the minima (or saddle point
    if( norm(g) < tol )
        M.converged = norm(g)
    end

    return x - α * g
end

# one liner stats call
get_stats(M::MyGradientDescent) = M.func_calls, M.grad_calls, M.iterations, M.converged

get_stats (generic function with 1 method)

In [5]:
# Example usage of the instrumented Version
f(x) = (x - 3)^2 # an example! root = 3
∇f(x) = 2 * (x - 3) # the derivative

M = MyGradientDescent(0.1)
x = 0.0  # Initial guess, we'll make this random soon

init!(M, f, ∇f(x), x)

iteration_limit = 3000
for i in 1:iteration_limit
    x = step!(M, f, ∇f, x)
end

@printf("Found solution: x ≈ %2.7f\n", x)
f_calls, g_calls, iterations, converged = M.func_calls, M.grad_calls, M.iterations, M.converged
@printf("Metrics: %d function calls, %d grad calls, %2.7f error in %d iterations \n", f_calls, g_calls, converged, iterations)

Found solution: x ≈ 2.9999996
Metrics: 0 function calls, 71 grad calls, 0.0000010 error in 71 iterations 


### Problem 1b

Now we'll create a custom version of the conjugate gradient implementation presented in the book. Just like before, we'll start with the version the book lays out!

In [6]:
# K&W algorithm 4.2 Code! Word for word, no modifications 
function backtracking_line_search(f, ∇f, x, d, α; p=0.5, β=1e-4)
    y, g = f(x), ∇f(x)
    
    while f(x + α*d) > y + β*α*(g⋅d)
        α *= p
    end
    
    return α
end


# K&W algorithm 5.2 Code! 
# Modified to use backtracking / Approximate line search instead
# of regular line search based on univariate minimization
# + Some comments for me to understand it later
mutable struct ConjugateGradientDescent <: DescentMethod
    d # previous gradient direction
    g # previous gradient vector
end

function init!(M::ConjugateGradientDescent, f, ∇f, x)
    M.g = ∇f(x)
    M.d = -M.g
    return M
end

function step!(M::ConjugateGradientDescent, f, ∇f, x)
    d, g = M.d, M.g
    g′ = ∇f(x)
    β = max(0, dot(g′, g′-g)/dot(g, g))
    d′ = -g′ + β*d
    x′ = x + backtracking_line_search(f, ∇f, x, d′, 10) * d′  # modified to use backtracking line search !!
    M.d, M.g = d′, g′
    return x′
end

step! (generic function with 3 methods)

Just like before, we'll run this example to show it could be utilized.

#### Explaination

Just like before, we're reusing the interface definition from earlier $DescentMethod$. The struct was updated to be mutable, since it'll hold the previous calculations. The $init$ function was also change to calculate an initial value for the struct. This assigns the initial direction and gradient vector itself.

A small modification was made to the step function to handle the backtracking/approximate line search instead of the originial line searched based on univariate optimization. This meant calculating the change in step size, and applying it manually instead of reassigning step size directly. (Since backtracking line search returns how we should modify the step size, rather than the step size directly).

Everything was unchanged from the book's implementation.

In [7]:
# Example usage of the K&W algorithm 5.2 Code w/ backtracking line search! 
f(x) = (x - 3)^2 # an example! root = 3
∇f(x) = 2 * (x - 3) # the derivative

M = ConjugateGradientDescent(0.1, 0.1)
x = 0.0  # Initial guess, we'll make this random soon

init!(M, f, ∇f, x)

for i in 1:20
    x = step!(M, f, ∇f, x)
    @printf("Iteration %d: x = %2.7f, f(x) = %2.7f\n", i, x, f(x))
end

@printf("Found solution: x ≈ %2.7f\n", x)

Iteration 1: x = 3.7500000, f(x) = 0.5625000
Iteration 2: x = 3.7500000, f(x) = 0.5625000
Iteration 3: x = 2.8125000, f(x) = 0.0351562
Iteration 4: x = 2.8125000, f(x) = 0.0351562
Iteration 5: x = 3.0468750, f(x) = 0.0021973
Iteration 6: x = 3.0468750, f(x) = 0.0021973
Iteration 7: x = 2.9882812, f(x) = 0.0001373
Iteration 8: x = 2.9882812, f(x) = 0.0001373
Iteration 9: x = 3.0029297, f(x) = 0.0000086
Iteration 10: x = 3.0029297, f(x) = 0.0000086
Iteration 11: x = 2.9992676, f(x) = 0.0000005
Iteration 12: x = 2.9992676, f(x) = 0.0000005
Iteration 13: x = 3.0001831, f(x) = 0.0000000
Iteration 14: x = 3.0001831, f(x) = 0.0000000
Iteration 15: x = 2.9999542, f(x) = 0.0000000
Iteration 16: x = 2.9999542, f(x) = 0.0000000
Iteration 17: x = 3.0000114, f(x) = 0.0000000
Iteration 18: x = 3.0000114, f(x) = 0.0000000
Iteration 19: x = 2.9999971, f(x) = 0.0000000
Iteration 20: x = 2.9999971, f(x) = 0.0000000
Found solution: x ≈ 2.9999971


#### Instrumentation!

Again, just like before, we'll adjust the structure to keep track of function calls, gradient calls, and iteration count! Since these will ultimately be used to evaluate the gradient methods against each other.

In [8]:
# K&W algorithm 5.2 Code! 
# Modified to use backtracking / Approximate line search instead
# of regular line search based on univariate minimization
# Also modified to keep track of function calls via structure memebers
mutable struct MyConjugateGradientDescent <: DescentMethod
    d::Vector{Float64} # previous gradient direction
    g::Vector{Float64} # previous gradient vector
    
    # instrumention stuff
    func_calls::Int
    grad_calls::Int
    iterations::Int
    converged::Float64

    # Constructor with optional instrumentation fields defaulting to zero/false
    function MyConjugateGradientDescent()
        new([], [], 0, 0, 0, 0.0)
    end
end

# K&W algorithm 4.2 Code!
# Modified to keep track of function calls, and gradient calls
# using a common struct that's passed in!
function my_backtracking_line_search(M::MyConjugateGradientDescent, f, ∇f, x, d, α; p=0.5, β=1e-4)
    y, g = f(x), ∇f(x)
    M.func_calls += 1
    M.grad_calls += 1
    
    while f(x + α*d) > y + β*α*dot(g,d)
        M.func_calls += 1
        α *= p
    end
    
    return α
end

function init!(M::MyConjugateGradientDescent, f, ∇f, x)
    #zero out instrumentation if reusing M
    M.func_calls = 0
    M.grad_calls = 0
    M.iterations = 0
    M.converged = 0.0
    
    # regular Init for conjugate gradient descent
    M.g = ∇f(x)
    M.grad_calls += 1
    M.d = -M.g
    return M
end

function step!(M::MyConjugateGradientDescent, f, ∇f, x; tol=1e-6, max_step_size=10)
    
    # check for early exit
    if( M.converged > 0 )
        return x
    end
    
    # let's do an iteration!
    M.iterations += 1
    
    d, g = M.d, M.g
    g′ = ∇f(x)
    M.grad_calls += 1
    
    β = max(0, dot(g′, g′-g)/dot(g, g))
    d′ = -g′ + β*d
    x′ = x + my_backtracking_line_search(M, f, ∇f, x, d′, max_step_size) * d′  # modified to use backtracking line search !!
    M.d, M.g = d′, g′
    
    # tolerance check, maybe we're done! If the magnitude / norm of the gradient is small
    # we're probably in the minima (or saddle point)
    if( norm(M.g) < tol )
        M.converged = norm(M.g)
    end
    
    return x′
end

step! (generic function with 4 methods)

### Problem 1c

Experiment time! Definte the rosenbrock function and it's gradient - and we'll use the booth function like the last homework. Note, these are normally 2D problems, but to extend these problems to more dimensions, we effectively treat each variable pair wise. So, $x_1$, $x_2$ are paired, $x2$, $x3$ are paired, etc until $x_n$ is reached. In effect, sequential variables are coupled together. And the results are summed together to evaluate the function. 

Note: In order to use ForwardDiff package, type information must be removed from parameters. So preallocating array isn't possible any the type is any.

In [9]:
# n-dimensional Rosenbrock! (as specified by assignment)
function rosenbrock_n(x; a = 1.0, b = 5.0, n = 10)
    # Compute the sum
    result = 0.0
    # we end at n-1, because we need the last entry to do n-1th entry
    for i in 1:(n - 1)
        result += (a - x[i])^2 + b * (x[i + 1] - x[i]^2)^2
    end
    return result
end

function rosenbrock_n_gradient(x; a = 1.0, b = 5.0, n = 10)
    grad = Float64[]

    # Special case for the first element
    push!( grad, -2 * (a - x[1]) + (-4) * b * x[1] * ( x[2] - x[1]^2) )
    
    # Compute the gradient for the 2:n-1 elements
    for i in 2:(n-1)
        push!( grad, ( -2*(a - x[i]) + 2 * b * (x[i+1]-x[i]^2) * -2 * x[i] ) + 2 * b * (x[i] - x[i-1]^2) )
    end

    # Special case for the last element (n-th element)
    push!( grad, 2 * b * (x[n] - x[n-1]^2) )

    return grad
end


rosenbrock_n_gradient (generic function with 1 method)

In [10]:
function booths_n(x; n = 10)
    result = 0.0
    # same rationale for ending at n-1, we need the nth element for n-1
    for i in 1:(n - 1)
        result += ((x[i] + 2*x[i+1] - 7)^2 + (2*x[i] + x[i+1] - 5)^2)
    end
    return result
end

function booths_n_gradient(x; n = 10)
    grad = Float64[]
    
    # Special case for the first element
    push!( grad, 2*(x[1]+2*x[2]-7) + 4*(2*x[1] + x[2] - 5) )
    
    # Compute the gradient for the 2:n-1 elements
    for i in 2:(n-1)
        push!( grad, 2*(x[i]+2*x[i+1]-7) + 4*(2*x[i] + x[i+1] - 5) + 4*(x[i-1] + 2*x[i] - 7) + 2*(2*x[i-1] + x[i] - 5) )
    end

    # Special case for the last element (n-th element)
    push!( grad, 4*(x[n-1] + 2*x[n] - 7) + 2*(2*x[n-1] + x[n] - 5) ) 
    
    return grad
end

booths_n_gradient (generic function with 1 method)

In [11]:
# check gradients using auto diff, since that has no error!
x = [i/5 for i in 1:10]
@printf("L2 error of rosenbrock: %f \n", sum((rosenbrock_n_gradient(x) - ForwardDiff.gradient(rosenbrock_n, x)).^2))
@printf("L2 error of booth: %f\n", sum((booths_n_gradient(x) - ForwardDiff.gradient(booths_n, x)).^2))

L2 error of rosenbrock: 0.000000 
L2 error of booth: 0.000000


## Experimental Set Up

With both functions and their gradients defined, we can begin to set up the experiment. The stopping criteria here is gradient convergence. The idea being that if we've reached a minima, the norm of the gradient vector should be tiny. I opted for this definition of convergence since it's possible that there may be little change in experimental variables but massive changes in slope (which means we're still optimizing!). 

To do the actual experiment, we'll generate 100 different starting points, varying from -10 to 10. These will all be randomly generated. In order to report error, I need to have an answer to compare against. I'll use my code from the previous assignment, but updating it to have a parameter for the number of dimensions per random vector.

In [12]:
function generate_initial_points(left_limit=-10.0, right_limit=10.0, samples=100, dims=10)
    # Generate 100 random real-valued starting points as specified
    initial_points = [] 

    for i in 1:samples # defaults to 100 samples
        this_sample = []
        
        for j in 1:dims
            # y is uniformly picked in [-10.0, 10.0] by default
            # note, rand is already uniform, just need to cap the limits
            y = rand(Uniform(left_limit, right_limit))
            # choose + or - with equal probability, we'll use bitrand (50/50)
            sign = bitrand() == 0 ? -1 : 1
            x = sign * exp(y)
            push!(this_sample, x)
        end
        
        push!(initial_points, this_sample)
    end

    return initial_points
end

generate_initial_points (generic function with 5 methods)

In [13]:
generate_initial_points()

100-element Vector{Any}:
 Any[56.52559752390986, 0.0019517842076887226, 0.0343311788946354, 439.95881215660245, 9.329145648901044e-5, 4.650768343637638, 0.0033941816256434208, 0.5029550031940542, 0.6544952296363804, 7.517280410403966]
 Any[8.036359348702595, 0.046619560977479754, 1.180421446592528, 198.3336564463679, 16.426684380533267, 0.005824155868490791, 876.6540339947347, 0.00019011354496189262, 9936.535212502227, 0.001398343023486806]
 Any[9.0235143722527, 0.5948571442761463, 62.308639338771066, 0.00013834331597270088, 2938.1642755478933, 80.37155958699849, 7.28395481068361e-5, 0.00023069013238338583, 0.013061140290287068, 15.449331104461034]
 Any[1.9280229238917257, 69.71358449889952, 0.04723373382924827, 12.272310080228438, 4759.96224883146, 3369.5017971046236, 0.009137921397509007, 3841.601994483109, 0.040305692495622596, 0.014275271909965147]
 Any[0.025204234479678247, 3542.595474580546, 0.4626233323760416, 6.0052427577234036e-5, 2823.5948532855246, 0.08572935429305956, 0.009

In [14]:
# Lists to store results for entire run
results_grad_rosenbrock = []
results_conj_rosenbrock = []
results_grad_booths = []
results_conj_booths = []

# Generate initial points for each function we'll be testing
initial_points_rosenbrock = generate_initial_points()
initial_points_booths = generate_initial_points()

# Create the Models we'll use (our algos)
gradient = MyGradientDescent(0.5)
conjugate = MyConjugateGradientDescent()

# Get our solutions
rosenbrock_opt_res = optimize(rosenbrock_n, convert(Vector{Float32}, initial_points_rosenbrock[1]), LBFGS(); autodiff = :forward)
rosenbrock_opt_min_x = Optim.minimizer(rosenbrock_opt_res)

# Get our solutions
booths_opt_res = optimize(booths_n, convert(Vector{Float32}, initial_points_booths[1]), LBFGS(); autodiff = :forward)
booths_opt_min_x = Optim.minimizer(booths_opt_res)

# Run the optimizer for each function! Using each model
for (M, result_containers) in zip([gradient, conjugate], [(results_grad_rosenbrock,results_grad_booths) , (results_conj_rosenbrock, results_conj_booths)])
    
    # Lists to store results
    results_rosenbrock = []
    results_booths = []

    for (targets, analytical_solution, points, results) in zip(
            [(rosenbrock_n, rosenbrock_n_gradient), (booths_n, booths_n_gradient)], 
            [rosenbrock_opt_min_x, booths_opt_min_x], 
            [initial_points_rosenbrock, initial_points_booths], 
            [results_rosenbrock, results_booths])    

        (func, func_grad) = targets

        for x in points
            start_time = time()
            init!(M, func, func_grad, x)

            iteration_limit = 1000
            for i in 1:iteration_limit
                x = step!(M, func, func_grad, x)
                
                # Early Exit if we have disappearing gradient or exploding gradient
                if any(isnan, x) || any(isinf, x)
                    break  # stop processing samples!
                end
            end

            elapsed_time = time() - start_time
            f_calls, g_calls, iterations, converged = M.func_calls, M.grad_calls, M.iterations, M.converged
            
            absolute_error = norm(x - analytical_solution, Inf)  # Infinity norm for max absolute error
            
            push!(results, (absolute_error, converged, f_calls + g_calls, elapsed_time))
        end
    end
    
    (rosen, booth) = result_containers
    
    append!(rosen, results_rosenbrock)
    append!(booth, results_booths)
    
end

In [15]:
results_grad_rosenbrock

100-element Vector{Any}:
 (Inf, 0.0, 4, 0.17285895347595215)
 (NaN, 0.0, 5, 7.891654968261719e-5)
 (Inf, 0.0, 5, 1.5974044799804688e-5)
 (NaN, 0.0, 5, 1.1920928955078125e-5)
 (NaN, 0.0, 5, 1.0967254638671875e-5)
 (NaN, 0.0, 5, 1.0967254638671875e-5)
 (Inf, 0.0, 4, 9.059906005859375e-6)
 (Inf, 0.0, 4, 1.0013580322265625e-5)
 (Inf, 0.0, 4, 8.821487426757812e-6)
 (NaN, 0.0, 5, 1.1920928955078125e-5)
 (NaN, 0.0, 5, 1.0013580322265625e-5)
 (NaN, 0.0, 5, 1.0013580322265625e-5)
 (Inf, 0.0, 5, 1.0013580322265625e-5)
 ⋮
 (Inf, 0.0, 4, 1.0013580322265625e-5)
 (Inf, 0.0, 4, 9.059906005859375e-6)
 (Inf, 0.0, 4, 1.0967254638671875e-5)
 (Inf, 0.0, 4, 9.059906005859375e-6)
 (NaN, 0.0, 5, 1.2159347534179688e-5)
 (NaN, 0.0, 5, 1.0013580322265625e-5)
 (Inf, 0.0, 4, 9.059906005859375e-6)
 (NaN, 0.0, 5, 1.0013580322265625e-5)
 (Inf, 0.0, 4, 1.0013580322265625e-5)
 (Inf, 0.0, 4, 1.0013580322265625e-5)
 (Inf, 0.0, 4, 9.059906005859375e-6)
 (Inf, 0.0, 4, 1.0013580322265625e-5)

Process the results into the table required by the assignment!

In [16]:
function summarize_results(results; nan_replacement=nothing)
    # Initialize arrays to collect numerical data
    absolute_errors = Float64[]
    convergences = Float64[]
    times = Float64[]
    evaluations = Int[]

    for r in results
        # Extract data from r[1], r[2], and r[4]
        # They may be numbers or iterables (tuples/arrays)

        # Process r[1]: absolute_errors
        data_r1 = r[1]
        values_r1 = isa(data_r1, Number) ? [data_r1] : collect(data_r1)

        # Process r[2]: convergences
        data_r2 = r[2]
        values_r2 = isa(data_r2, Number) ? [data_r2] : collect(data_r2)

        # Process r[4]: times
        data_r4 = r[4]
        values_r4 = isa(data_r4, Number) ? [data_r4] : collect(data_r4)

        # Process r[3]: evaluations (assumed to be valid)
        evaluations_r = r[3]

        # Handle NaNs
        if isnothing(nan_replacement)
            # Skip NaN values
            values_r1 = [v for v in values_r1 if !isnan(v)]
            values_r2 = [v for v in values_r2 if !isnan(v)]
            values_r4 = [v for v in values_r4 if !isnan(v)]
        else
            # Replace NaNs with the specified replacement value
            values_r1 = [isnan(v) ? nan_replacement : v for v in values_r1]
            values_r2 = [isnan(v) ? nan_replacement : v for v in values_r2]
            values_r4 = [isnan(v) ? nan_replacement : v for v in values_r4]
        end

        # Append numerical values to arrays
        append!(absolute_errors, values_r1)
        append!(convergences, values_r2)
        append!(times, values_r4)
        push!(evaluations, evaluations_r)
    end

    # Compute statistics, handling empty lists gracefully
    mean_absolute_error = isempty(absolute_errors) ? NaN : mean(absolute_errors)
    std_absolute_error = isempty(absolute_errors) ? NaN : std(absolute_errors)
    mean_convergence = isempty(convergences) ? NaN : mean(convergences)
    std_convergence = isempty(convergences) ? NaN : std(convergences)
    mean_evaluations = mean(evaluations)
    std_evaluations = std(evaluations)
    mean_time = isempty(times) ? NaN : mean(times) * 1e3  # Convert to milliseconds
    std_time = isempty(times) ? NaN : std(times) * 1e3    # Convert to milliseconds

    return (mean_absolute_error, std_absolute_error,
            mean_convergence, std_convergence,
            mean_evaluations, std_evaluations,
            mean_time, std_time)
end


# Summarize results for each function
summary_rosen_grad = summarize_results(results_grad_rosenbrock)
summary_rosen_conj = summarize_results(results_conj_rosenbrock)
summary_booth_grad = summarize_results(results_grad_booths)
summary_booth_conj = summarize_results(results_conj_booths)

# Print results in a 4x4 table
# Create a Table to display results as a table
using PrettyTables

headers = ["Function", "Absolute Error (mean ± std)", "Convergence (mean ± std)", "Evaluations (mean ± std)", "Time (ms) (mean ± std)"]
fdata = hcat("Rosen(GD)", @sprintf("%.2e ± %.2e", summary_rosen_grad[1], summary_rosen_grad[2]), @sprintf("%.2e ± %.2e", summary_rosen_grad[3], summary_rosen_grad[4]), @sprintf("%.2f ± %.2f", summary_rosen_grad[5], summary_rosen_grad[6]), @sprintf("%.2f ± %.2f", summary_rosen_grad[7], summary_rosen_grad[8]))
gdata = hcat("Rosen(CD)", @sprintf("%.2e ± %.2e", summary_rosen_conj[1], summary_rosen_conj[2]), @sprintf("%.2e ± %.2e", summary_rosen_conj[3], summary_rosen_conj[4]), @sprintf("%.2f ± %.2f", summary_rosen_conj[5], summary_rosen_conj[6]), @sprintf("%.2f ± %.2f", summary_rosen_conj[7], summary_rosen_conj[8]))
hdata = hcat("Booth(GD)", @sprintf("%.2e ± %.2e", summary_booth_grad[1], summary_booth_grad[2]), @sprintf("%.2e ± %.2e", summary_booth_grad[3], summary_booth_grad[4]), @sprintf("%.2f ± %.2f", summary_booth_grad[5], summary_booth_grad[6]), @sprintf("%.2f ± %.2f", summary_booth_grad[7], summary_booth_grad[8]))
jdata = hcat("Booth(CD)", @sprintf("%.2e ± %.2e", summary_booth_conj[1], summary_booth_conj[2]), @sprintf("%.2e ± %.2e", summary_booth_conj[3], summary_booth_conj[4]), @sprintf("%.2f ± %.2f", summary_booth_conj[5], summary_booth_conj[6]), @sprintf("%.2f ± %.2f", summary_booth_conj[7], summary_booth_conj[8]))


pretty_table(vcat(fdata, gdata, hdata, jdata), header=headers)

┌───────────┬─────────────────────────────┬──────────────────────────┬──────────────────────────┬────────────────────────┐
│  Function │ Absolute Error (mean ± std) │ Convergence (mean ± std) │ Evaluations (mean ± std) │ Time (ms) (mean ± std) │
├───────────┼─────────────────────────────┼──────────────────────────┼──────────────────────────┼────────────────────────┤
│ Rosen(GD) │                   Inf ± NaN │      0.00e+00 ± 0.00e+00 │              4.43 ± 0.52 │           1.74 ± 17.28 │
│ Rosen(CD) │         1.02e+03 ± 2.99e+03 │      6.40e-07 ± 3.98e-07 │      12603.31 ± 11099.33 │           5.81 ± 25.64 │
│ Booth(GD) │                   Inf ± NaN │      0.00e+00 ± 0.00e+00 │            250.65 ± 0.78 │            0.85 ± 5.64 │
│ Booth(CD) │         2.94e-07 ± 1.14e-08 │      6.38e-07 ± 1.96e-07 │          999.31 ± 154.22 │           2.32 ± 14.21 │
└───────────┴─────────────────────────────┴──────────────────────────┴──────────────────────────┴────────────────────────┘


We didn't achieve convergence at all with Gradient Descent (GD). While with Conjugate Descent, we got convergence, at the cost of much much more computation. Note, we experimenced disappearing gradients with the regular GD algorithm, resuling in NaNs or Infs when doing the derviate and applying the learning rate update.

## Problem 2, An advanced optimizer! Enhancing GD

Since I'm a grad student, I'll have to do problem 2! For this problem, I'll go ahead and implement one of the various algorithms in Chapter 5 of the book. Basically, all the remaining methods add momentum. I'll go with Nesterov Momentum since it's a bit out of vogue. It's Algorithm 5.4 from K&W

In [17]:
# Taken from Algorithm 5.4 in K&W, no modifications made

mutable struct NesterovMomentum <: DescentMethod
    α # learning rate
    β # momentum decay
    v # momentum
end

function init!(M::NesterovMomentum, f, ∇f, x)
    M.v = zeros(length(x))
    return M
end

function step!(M::NesterovMomentum, f, ∇f, x)
    α, β, v = M.α, M.β, M.v
    v[:] = β*v - α*∇f(x + β*v)
    return x + v
end

step! (generic function with 5 methods)

Like the other models before, we'll go ahead and instrument this up to keep track of various parameters.

In [18]:
# Taken from Algorithm 5.4 in K&W
# Added instrumentation! For function evals and for iteration count

mutable struct MyNesterovMomentum <: DescentMethod
    α # learning rate
    β # momentum decay
    v # momentum
    
    # instrumention stuff
    func_calls::Int
    grad_calls::Int
    iterations::Int
    converged::Float64

    # Constructor with optional instrumentation fields defaulting to zero/false
    function MyNesterovMomentum(α, β)
        new(α, β, [], 0, 0, 0, 0.0)
    end
end

function init!(M::MyNesterovMomentum, f, ∇f, x)
    M.v = zeros(length(x))
    
    # instrumention stuff, if we reuse M
    func_calls = 0
    grad_calls = 0
    iterations = 0
    converged = 0.0
    
    return M
end

function step!(M::MyNesterovMomentum, f, ∇f, x)
    if( M.converged > 0.0 )
        return x
    end
    
    M.iterations += 1
    
    α, β, v = M.α, M.β, M.v
    v[:] = β*v - α*∇f(x + β*v)
    M.grad_calls += 1
    
    return x + v
end

# one liner stats call
get_stats(M::MyNesterovMomentum) = M.func_calls, M.grad_calls, M.iterations, M.converged

get_stats (generic function with 2 methods)

Let's run the same experiment! But this time with our brand new Optimizer!

In [25]:
# Lists to store results for entire run
results_nest_rosenbrock = []
results_nest_booths = []

# Generate initial points for each function we'll be testing
initial_points_rosenbrock = generate_initial_points()
initial_points_booths = generate_initial_points()

# Create the Models we'll use (our algos)
nest = MyNesterovMomentum(0.0001, 0.5)

# Get our solutions
rosenbrock_opt_res = optimize(rosenbrock_n, convert(Vector{Float32}, initial_points_rosenbrock[1]), LBFGS(); autodiff = :forward)
rosenbrock_opt_min_x = Optim.minimizer(rosenbrock_opt_res)

# Get our solutions
booths_opt_res = optimize(booths_n, convert(Vector{Float32}, initial_points_booths[1]), LBFGS(); autodiff = :forward)
booths_opt_min_x = Optim.minimizer(booths_opt_res)

# Lists to store results
results_rosenbrock = []
results_booths = []

for (targets, analytical_solution, points, results) in zip(
        [(rosenbrock_n, rosenbrock_n_gradient), (booths_n, booths_n_gradient)], 
        [rosenbrock_opt_min_x, booths_opt_min_x], 
        [initial_points_rosenbrock, initial_points_booths], 
        [results_rosenbrock, results_booths])    

    (func, func_grad) = targets

    for x in points
        start_time = time()
        init!(nest, func, func_grad, x)

        iteration_limit = 1000
        for i in 1:iteration_limit
            x = step!(nest, func, func_grad, x)

            # did we converge?
            if( norm(func_grad(x)) < 1e-6 )
                nest.converged = norm(func_grad(x))
            end
            
            # Early Exit if we have disappearing gradient or exploding gradient
            if any(isnan, x) || any(isinf, x)
                break  # stop processing samples!
            end
            
        end

        elapsed_time = time() - start_time
        f_calls, g_calls, iterations, converged = nest.func_calls, nest.grad_calls, nest.iterations, nest.converged

        absolute_error = norm(x - analytical_solution, Inf)  # Infinity norm for max absolute error

        push!(results, (absolute_error, converged, f_calls + g_calls, elapsed_time))
    end
end

In [26]:
# Summarize results for each function
summary_rosen_grad = summarize_results(results_rosenbrock)
summary_rosen_conj = summarize_results(results_booths)

# Print results in a 4x4 table
# Create a Table to display results as a table
using PrettyTables

headers = ["Function", "Absolute Error (mean ± std)", "Convergence (mean ± std)", "Evaluations (mean ± std)", "Time (ms) (mean ± std)"]
fdata = hcat("Rosen(ND)", @sprintf("%.2e ± %.2e", summary_rosen_grad[1], summary_rosen_grad[2]), @sprintf("%.2e ± %.2e", summary_rosen_grad[3], summary_rosen_grad[4]), @sprintf("%.2f ± %.2f", summary_rosen_grad[5], summary_rosen_grad[6]), @sprintf("%.2f ± %.2f", summary_rosen_grad[7], summary_rosen_grad[8]))
gdata = hcat("Booths(ND)", @sprintf("%.2e ± %.2e", summary_rosen_conj[1], summary_rosen_conj[2]), @sprintf("%.2e ± %.2e", summary_rosen_conj[3], summary_rosen_conj[4]), @sprintf("%.2f ± %.2f", summary_rosen_conj[5], summary_rosen_conj[6]), @sprintf("%.2f ± %.2f", summary_rosen_conj[7], summary_rosen_conj[8]))

pretty_table(vcat(fdata, gdata), header=headers)

┌────────────┬─────────────────────────────┬──────────────────────────┬──────────────────────────┬────────────────────────┐
│   Function │ Absolute Error (mean ± std) │ Convergence (mean ± std) │ Evaluations (mean ± std) │ Time (ms) (mean ± std) │
├────────────┼─────────────────────────────┼──────────────────────────┼──────────────────────────┼────────────────────────┤
│  Rosen(ND) │                   Inf ± NaN │      0.00e+00 ± 0.00e+00 │          258.36 ± 148.49 │            0.02 ± 0.01 │
│ Booths(ND) │         1.32e+03 ± 1.38e+03 │      0.00e+00 ± 0.00e+00 │      51011.00 ± 29011.49 │            1.77 ± 0.75 │
└────────────┴─────────────────────────────┴──────────────────────────┴──────────────────────────┴────────────────────────┘


Very simliar behavior to Gradient Descent for the 10 dimensional Rosenbrock. It even took more iterations than regular gradient descent to get to the same vanishing or exploding gradient problem. Meaning, regular Gradient Descent was able to realize it could never converge quicker. However, this could also mean that Nesterov Momentum was able to actually follow the gradients for a longer duration.

What's interesting is Nesterov Momentum could keep iterating long enough to have somewhat real errors with infinite norm! And it did it much faster (about 1 ms faster) than Conjugate descent on average. But it had an order of magnitude more evaluations to get the same result. And the error is quite large when comparing the two. However, using the same convergence metric as earlier (if the norm of the gradient gets small) it's clear that the gradient is still disappearing or exploding!

To really get anything useful out of Gradient Descent or Nesterov Momentum Descent, I think I need to tune the hyper parameters a bit more. For example, messing with the learning rate of Gradient Descent (and the line search), I was able to get to some minima. Another possible route is constraining the design points - getting them closer to where the actual minima is could really help these methods. 